In [15]:
import pyaes

#################################STAGE 1###############################################

# A 128 bit (16 byte) key
key = bytes.fromhex('00' * 16)
print(len(key))
# For some modes of operation we need a random initialization vector
# of 16 bytes
iv = "CAFEBABEFACEDBADDECAF888"
iv = '00' * 12
iv = bytes.fromhex(iv)

aes = pyaes.AESModeOfOperationECB(key)
plaintext = bytes.fromhex("00"* 16)
H = ciphertext = aes.encrypt(plaintext)
H.hex()

16


'66e94bd4ef8a2c3b884cfa59ca342b2e'

In [10]:
b1 = "B83B5337 08BF535D 0AA6E529 80D53B78"
c = bytes.fromhex(b1);c.hex()
c == ciphertext

False

In [16]:
#################################STAGE 2 - 3###############################################

# X is the plaintext
# ICB is the increased J0
# Key is key
pt = bytes.fromhex('D9313225 F88406E5 A55909C5 AFF5269A 86A7A953 1534F7DA 2E4C303D 8A318A72 1C3C0C95 95680953 2FCF0E24 49A6B525 B16AEDF5 AA0DE657 BA637B39 1AAFD255')
aad = bytes.fromhex('3AD77BB4 0D7A3660 A89ECAF3 2466EF97 F5D3D585 03B9699D E785895A 96FDBAAF 43B1CD7F 598ECE23 881B00E3 ED030688 7B0C785E 27E8AD3F 82232071 04725DD4')
# key = '00' * 16

pt = bytes.fromhex('00' * 16)
aad = bytes.fromhex('00' * 16)

j0 = iv + b'00' * 3 + b'01'
import math
def xor(a, b):
    return bytes([aa^bb for aa, bb in zip(a, b)])
        
def gctr(icb, key, x):
    cb = []
    for i in range(16):
        cb.append(iv + b'\x00' * 3 + bytes([i]))
    if x == "":
        return ''
    y = []
    n = math.ceil(len(x)/16)
#     print(cb[2])
    for j in range(n):
        cb_ = cb[j + 2]
        ciphed = aes.encrypt(cb_)
        y.append(xor(x[j* 16 : j* 16 + 16], ciphed))
    return y
    
yy = gctr(j0, key, pt)
C= b''
for y in yy:
    C = C+ y

In [21]:
#################################STAGE 4 5 6###############################################
# u = aad + C + 3 * b'\x00' + bytes()
def shiftright(a):
    leng = len(a)
    carry = 0
    res = []
    for i in range(leng):
        swap = a[i] % 2
        res.append(a[i] // 2 + carry * (pow(2, 7)))
        carry = swap
    return bytes(res)
        
def gctr1(icb, key, x):
    cb = []
    for i in range(16):
        cb.append(iv + b'\x00' * 3 + bytes([i]))
    if x == "":
        return ''
    y = []
    n = math.ceil(len(x)/16)
#     print(cb[2])
    for j in range(n):
        cb_ = cb[j + 1]
        ciphed = aes.encrypt(cb_)
        y.append(xor(x[j* 16 : j* 16 + 16], ciphed))
    return y

def hashmul(x, y):
    z = b'\x00' * 16 
    v = y
    ele = b'\xe1' + b'\x00' * 15
    
    for i in x:
        num = i
        for j in range(8):
            div = pow(2, 7 - j)
            here = num // div
            num = num % div
            
            if here == 1:
                z = xor(z, v)
            if v[-1] % 2 == 0:
                v = shiftright(v)
            else:
                v = shiftright(v)
                v = xor(v, ele)
    return z
y1 = b'\x00' * 16
xx = aad + C + b'\x00' * 7 + b'\x80' +  b'\x00' * 7 + b'\x80' 

for i in range(3):
    y1 = hashmul(xor(xx[i * 16:i * 16 +16], y1), H)
S = y1
t = gctr1(j0, key, S)
T = t[0]

In [26]:
print('S is\n\t\t', S.hex())
print('\nC is\t\t')
for i in range(1):
    print('\t\t', C[i * 16:i * 16 +16].hex())
print('\nTag is\n','\t\t',T.hex())

S is
		 8aacacf4e1ce07662a0eae0a3c9cc30d

C is		
		 0388dace60b6a392f328c2b971b2fe78

Tag is
 		 d24e503a1bb037071c71b35d987b8657
